In [1]:
import pandas as pd
import yaml
import os
import re
import numpy as np

Create cross validation dataframe for variation in

1. bert learning rate
2. model learning rate
3. preprocess
4. warmup

In [2]:
results_dir = os.path.join(os.getenv("DATA_DIR"),
                           "mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec19-21")
header = ["preprocess", "bert_lr", "model_lr", "warmup", "movie", "dev_lea", "train_lea", "best_epoch", "dev_scores",
          "train_scores"]
rows = []

for dir_ in os.listdir(results_dir):
    result_file1 = os.path.join(results_dir, dir_, "result.yaml")
    result_file2 = os.path.join(results_dir, dir_, "result2.yaml")
    if os.path.exists(result_file1):
        with open(result_file1) as f:
            result = yaml.load(f, Loader=yaml.FullLoader)
    elif os.path.exists(result_file2):
        with open(result_file2) as f:
            result = yaml.load(f, Loader=yaml.FullLoader)
    else:
        continue
    preprocess = result["preprocess"]
    bert_lr = result["bert_lr"]
    model_lr = result["coref_lr"]
    warmup = result["warmup_steps"]
    movie = result["test_movie"]
    dev_lea = result["dev_metric"]["span"]["lea"]["f1"]
    epoch = result["best_epoch"]
    dev_scores = result["dev_scores"]
    train_scores = result["train_scores"]
    train_lea = train_scores[epoch - 1]
    rows.append([preprocess, bert_lr, model_lr, warmup, movie, dev_lea, train_lea, epoch, dev_scores, train_scores])

df = pd.DataFrame(rows, columns=header)
print(df.shape)
print()

for col, dtype in zip(df.columns, df.dtypes):
    print(f"{col} ({dtype})")

(648, 10)

preprocess (object)
bert_lr (float64)
model_lr (float64)
warmup (float64)
movie (object)
dev_lea (float64)
train_lea (float64)
best_epoch (int64)
dev_scores (object)
train_scores (object)


In [3]:
df.to_csv(os.path.join(os.getenv("DATA_DIR"), "mica_text_coref/movie_coref/results/coreference/cross_val_1.csv"),
          index=False)

Create cross validation dataframe for variation in

1. weight decay
2. dropout

In [9]:
results_dir = os.path.join(os.getenv("DATA_DIR"),
                           "mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec25")
dirs_with_no_result_file = []

for dir_ in os.listdir(results_dir):
    result_file = os.path.join(results_dir, dir_, "result.yaml")
    if not os.path.exists(result_file):
        dir__ = os.path.join(results_dir, dir_)
        print(dir__)
        dirs_with_no_result_file.append(dir__)

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec25/Dec25_12:40:52PM_prestige_excerpts_20332
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec25/Dec25_12:59:59PM_quiet_place_excerpts_20768
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec25/Dec25_02:01:20PM_dead_poets_society_excerpts_22143
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec25/Dec25_12:03:21PM_dead_poets_society_excerpts_19492
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec25/Dec25_03:24:52PM_zootopia_excerpts_23961
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec25/Dec25_03:19:32PM_dead_poets_society_excerpts_16196
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec25/Dec25_05:06:05PM_quiet_place_excerpts_26186
/proj/sbaruah/data/mica_text_core

In [19]:
header = ["weight_decay", "dropout", "movie", "dev_lea", "best_epoch", "dev_scores"]
rows = []
epoch_pattern = re.compile(r"Epoch = (\d+)\n"
                           r"dev:: loss=[0-9\.]+, metric:Word=[0-9\.]+, Span=([0-9\.]+), Character=[0-9\.]+")

for dir_ in dirs_with_no_result_file:
    log_file = os.path.join(dir_, "train.log")
    with open(log_file) as f:
        content = f.read()
    match = re.search(r"weight_decay\s+= ([0-9\.]+)", content)
    assert match is not None
    weight_decay = float(match.group(1))
    match = re.search(r"dropout\s+= ([0-9\.]+)", content)
    assert match is not None
    dropout = float(match.group(1))
    match = re.search(r"test_movie\s+= (\w+)", content)
    assert match is not None
    movie = match.group(1)
    
    epochs = []
    dev_scores = []

    for match in re.finditer(epoch_pattern, content):
        epochs.append(int(match.group(1)))
        dev_scores.append(float(match.group(2)))
    
    best_epoch_index = np.argmax(dev_scores).item()
    max_dev_score = dev_scores[best_epoch_index]
    best_epoch = epochs[best_epoch_index]

    print(dir_)
    print(weight_decay, dropout, movie, max_dev_score, best_epoch, dev_scores)
    rows.append([weight_decay, dropout, movie, max_dev_score, best_epoch, dev_scores])
    print()

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec25/Dec25_12:40:52PM_prestige_excerpts_20332
100.0 0.0 prestige 46.4 1 [46.4, 38.5, 17.0, 5.0, 1.8, 6.4]

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec25/Dec25_12:59:59PM_quiet_place_excerpts_20768
100.0 0.0 quiet_place 55.3 2 [41.0, 55.3, 32.7, 6.3, 0.0, 0.1, 0.0]

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec25/Dec25_02:01:20PM_dead_poets_society_excerpts_22143
100.0 0.2 dead_poets_society 41.9 1 [41.9, 28.2, 23.0, 3.4, 4.6, 4.3]

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec25/Dec25_12:03:21PM_dead_poets_society_excerpts_19492
100.0 0.0 dead_poets_society 37.9 1 [37.9, 0.9, 11.9, 3.7, 2.3, 1.7]

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec25/Dec25_03:24:52PM_zootopia_excerpts_23961
100.0 0.2 zootopia 61.3 1 [61.3, 57.1, 21

In [20]:
results_dir = os.path.join(os.getenv("DATA_DIR"),
                           "mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec25")

for dir_ in os.listdir(results_dir):
    result_file = os.path.join(results_dir, dir_, "result.yaml")
    if os.path.exists(result_file):
        with open(result_file) as f:
            result = yaml.load(f, Loader=yaml.FullLoader)
    else:
        continue
    weight_decay = result["weight_decay"]
    dropout = result["dropout"]
    movie = result["test_movie"]
    dev_lea = result["dev_metric"]["span"]["lea"]["f1"]
    epoch = result["best_epoch"]
    dev_scores = result["dev_scores"]
    rows.append([weight_decay, dropout, movie, dev_lea, epoch, dev_scores])

df = pd.DataFrame(rows, columns=header)
print(df.shape)
print()

for col, dtype in zip(df.columns, df.dtypes):
    print(f"{col} ({dtype})")

(240, 6)

weight_decay (float64)
dropout (float64)
movie (object)
dev_lea (float64)
best_epoch (int64)
dev_scores (object)


In [21]:
df.to_csv(os.path.join(os.getenv("DATA_DIR"), "mica_text_coref/movie_coref/results/coreference/cross_val_2.csv"),
          index=False)

Create cross validation dataframe for variation in

1. dev document len
2. dev overlap len

In [22]:
results_dir = os.path.join(os.getenv("DATA_DIR"),
                           "mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec26")
dirs_with_no_result_file = []

for dir_ in os.listdir(results_dir):
    result_file = os.path.join(results_dir, dir_, "result.yaml")
    if not os.path.exists(result_file):
        dir__ = os.path.join(results_dir, dir_)
        print(dir__)
        dirs_with_no_result_file.append(dir__)

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec26/Dec26_07:56:43AM_dead_poets_society_excerpts_21719
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec26/Dec26_05:13:00AM_avengers_endgame_excerpts_18334
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec26/Dec26_04:04:30PM_quiet_place_excerpts_31725
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec26/Dec26_11:11:20AM_john_wick_excerpts_25606
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec26/Dec26_07:14:21PM_zootopia_excerpts_3041
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec26/Dec26_01:33:32PM_prestige_excerpts_28638


In [25]:
results_dir = os.path.join(os.getenv("DATA_DIR"),
                           "mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec26")
header = ["dev_document_len", "dev_overlap_len", "movie", "dev_lea", "best_epoch", "dev_scores"]
rows = []

for dir_ in os.listdir(results_dir):
    result_file = os.path.join(results_dir, dir_, "result.yaml")
    if os.path.exists(result_file):
        with open(result_file) as f:
            result = yaml.load(f, Loader=yaml.FullLoader)
    else:
        continue
    dev_document_len = result["dev_document_len"]
    dev_overlap_len = result["dev_overlap_len"]
    movie = result["test_movie"]
    dev_lea = result["dev_metric"]["span"]["lea"]["f1"]
    epoch = result["best_epoch"]
    dev_scores = result["dev_scores"]
    rows.append([dev_document_len, dev_overlap_len, movie, dev_lea, epoch, dev_scores])

df = pd.DataFrame(rows, columns=header)
print(df.shape)
print()

for col, dtype in zip(df.columns, df.dtypes):
    print(f"{col} ({dtype})")

(204, 6)

dev_document_len (int64)
dev_overlap_len (int64)
movie (object)
dev_lea (float64)
best_epoch (int64)
dev_scores (object)


In [26]:
df.to_csv(os.path.join(os.getenv("DATA_DIR"), "mica_text_coref/movie_coref/results/coreference/cross_val_3.csv"),
          index=False)

Create cross validation dataframe for variation in (second run)

1. dev document len
2. dev overlap len

In [1]:
results_dir = os.path.join(os.getenv("DATA_DIR"),
                           "mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec28")
dirs_with_no_result_file = []

for dir_ in os.listdir(results_dir):
    result_file = os.path.join(results_dir, dir_, "result.yaml")
    if not os.path.exists(result_file):
        dir__ = os.path.join(results_dir, dir_)
        print(dir__)
        dirs_with_no_result_file.append(dir__)

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec28/Dec28_07:17:56AM_prestige_excerpts_25218
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec28/Dec28_04:34:13AM_prestige_excerpts_21742
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec28/Dec28_12:39:38AM_avengers_endgame_excerpts_16828
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec28/Dec28_03:17:09AM_quiet_place_excerpts_13198
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec28/Dec28_06:38:41AM_quiet_place_excerpts_24463
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec28/Dec28_08:00:39AM_zootopia_excerpts_26199
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec28/Dec28_02:08:56AM_prestige_excerpts_11679
/proj/sbaruah/data/mica_text_coref/movie_coref/results/

In [4]:
results_dir = os.path.join(os.getenv("DATA_DIR"),
                           "mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec28")
header = ["dev_document_len", "dev_overlap_len", "movie", "dev_lea", "best_epoch", "dev_scores"]
rows = []

for dir_ in os.listdir(results_dir):
    result_file = os.path.join(results_dir, dir_, "result.yaml")
    if os.path.exists(result_file):
        with open(result_file) as f:
            result = yaml.load(f, Loader=yaml.FullLoader)
    else:
        continue
    dev_document_len = result["dev_document_len"]
    dev_overlap_len = result["dev_overlap_len"]
    movie = result["test_movie"]
    dev_lea = result["dev_metric"]["span"]["lea"]["f1"]
    epoch = result["best_epoch"]
    dev_scores = result["dev_scores"]
    rows.append([dev_document_len, dev_overlap_len, movie, dev_lea, epoch, dev_scores])

df = pd.DataFrame(rows, columns=header)
print(df.shape)
print()

for col, dtype in zip(df.columns, df.dtypes):
    print(f"{col} ({dtype})")

(60, 6)

dev_document_len (int64)
dev_overlap_len (int64)
movie (object)
dev_lea (float64)
best_epoch (int64)
dev_scores (object)


In [5]:
macro_df = df.groupby(["dev_document_len", "dev_overlap_len"]).agg({"dev_lea": "mean", "movie": "count",
                                                                    "best_epoch": "mean"})
macro_df = macro_df[macro_df["movie"] == 6]
macro_df

dev_lea  movie  best_epoch
dev_document_len dev_overlap_len                              
5120             2048             75.227333      6   10.000000
8192             2048             76.658000      6    8.166667
                 3072             77.310667      6    6.666667
10240            2048             76.955833      6   10.500000
                 3072             78.889333      6    7.666667
                 4096             79.530167      6    7.666667
20480            2048             78.874833      6    8.666667
                 3072             79.197833      6    7.500000
                 4096             79.503500      6    8.333333
                 5120             79.380667      6    8.000000

In [6]:
old_df = pd.read_csv(os.path.join(os.getenv("DATA_DIR"),
                                  "mica_text_coref/movie_coref/results/coreference/cross_val_3.csv"), index_col=None)
new_df = pd.concat([old_df, df])
new_df.to_csv(os.path.join(os.getenv("DATA_DIR"), "mica_text_coref/movie_coref/results/coreference/cross_val_4.csv"),
              index=False)

Create cross validation data frame for hierarchical model
Variation in dev document len

In [2]:
results_dir = os.path.join(os.getenv("DATA_DIR"),
                           "mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_hi_Dec29")
dirs_with_no_result_file = []

for dir_ in os.listdir(results_dir):
    result_file = os.path.join(results_dir, dir_, "result.yaml")
    if not os.path.exists(result_file):
        dir__ = os.path.join(results_dir, dir_)
        print(dir__)
        dirs_with_no_result_file.append(dir__)
print()
print(f"{len(dirs_with_no_result_file)} directories have no result files")

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_hi_Dec29/Dec30_01:03:59AM_avengers_endgame_excerpts_hi_21693
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_hi_Dec29/Dec29_11:42:25PM_dead_poets_society_excerpts_hi_2374
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_hi_Dec29/Dec29_09:20:40PM_dead_poets_society_excerpts_hi_17243
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_hi_Dec29/Dec30_02:07:39AM_avengers_endgame_excerpts_hi_21361
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_hi_Dec29/Dec30_02:20:24AM_avengers_endgame_excerpts_hi_12698
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_hi_Dec29/Dec30_08:18:30AM_avengers_endgame_excerpts_hi_12692
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_hi_Dec29/Dec30_09:40:51AM_ave

In [5]:
for dir_ in dirs_with_no_result_file:
    log_file = os.path.join(dir_, "train.log")
    with open(log_file) as fr:
        content = fr.read()
    print(dir_)
    print(content.rstrip().endswith("Testing"))
    print("")

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_hi_Dec29/Dec30_01:03:59AM_avengers_endgame_excerpts_hi_21693
True

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_hi_Dec29/Dec29_11:42:25PM_dead_poets_society_excerpts_hi_2374
True

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_hi_Dec29/Dec29_09:20:40PM_dead_poets_society_excerpts_hi_17243
True

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_hi_Dec29/Dec30_02:07:39AM_avengers_endgame_excerpts_hi_21361
False

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_hi_Dec29/Dec30_02:20:24AM_avengers_endgame_excerpts_hi_12698
False

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_hi_Dec29/Dec30_08:18:30AM_avengers_endgame_excerpts_hi_12692
False

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val

In [11]:
results_dir = os.path.join(os.getenv("DATA_DIR"),
                           "mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_hi_Dec29")
epoch_pattern = re.compile(r"Epoch = (\d+)\n"
                           r"dev:: loss=[0-9\.]+, metric:Word=[0-9\.]+, Span=([0-9\.]+), Character=[0-9\.]+\n"
                           r"([0-9\.]+) GB GPU memory allocated during dev inference")
header = ["dev_document_len", "repk", "movie", "dev_lea", "best_epoch", "dev_scores", "dev_gpu_memory"]
rows = []

def search_regex(pattern, content):
    match = re.search(pattern, content)
    assert match is not None
    return match.group(1)

for dir_ in os.listdir(results_dir):
    result_file = os.path.join(results_dir, dir_, "result.yaml")
    log_file = os.path.join(results_dir, dir_, "train.log")
    with open(log_file) as fr:
        content = fr.read()
    if os.path.exists(result_file):
        with open(result_file) as f:
            result = yaml.load(f, Loader=yaml.FullLoader)
        dev_document_len = result["dev_document_len"]
        movie = result["test_movie"]
        dev_lea = result["dev_metric"]["span"]["lea"]["f1"]
        epoch = result["best_epoch"]
        dev_scores = result["dev_scores"]
        dev_gpu_memory = result["dev_gpu_memory"][epoch - 1]
    else:
        if re.search(r"\nTesting", content) is not None:
            dev_document_len = int(search_regex(r"dev_document_len\s+= (\d+)", content))
            movie = search_regex(r"test_movie\s+= (\w+)", content)
            dev_scores, epochs, dev_gpu_memory = [], [], []
            for match in re.finditer(epoch_pattern, content):
                dev_scores.append(float(match.group(2)))
                epochs.append(int(match.group(1)))
                dev_gpu_memory.append(float(match.group(3)))
            i = np.argmax(dev_scores)
            dev_lea = dev_scores[i]
            epoch = epochs[i]
            dev_gpu_memory = dev_gpu_memory[i]
        else:
            continue
    repk = int(search_regex(r"n_representative_mentions\s+= (\d+)", content))
    rows.append([dev_document_len, repk, movie, dev_lea, epoch, dev_scores, dev_gpu_memory])

df = pd.DataFrame(rows, columns=header)
print(df.shape)
print()

for col, dtype in zip(df.columns, df.dtypes):
    print(f"{col} ({dtype})")

(165, 7)

dev_document_len (int64)
repk (int64)
movie (object)
dev_lea (float64)
best_epoch (int64)
dev_scores (object)
dev_gpu_memory (float64)


In [12]:
macro_df = df.groupby(["dev_document_len", "repk"]).agg({"dev_lea": "mean", "movie": "count", "best_epoch": "mean",
                                                 "dev_gpu_memory": "mean"})
macro_df = macro_df[macro_df["movie"] == 6]
macro_df

dev_lea  movie  best_epoch  dev_gpu_memory
dev_document_len repk                                              
2048             1     65.067333      6    9.333333        8.860000
                 2     67.722833      6    8.000000       12.940000
                 3     68.848333      6    7.833333       19.486667
3072             1     66.021500      6    7.666667        8.533333
                 2     68.347333      6    9.000000       11.503333
                 3     70.927333      6    8.666667       16.506667
4096             1     67.065333      6    9.166667        8.381667
                 2     70.519833      6   10.500000       10.873333
                 3     70.619833      6    7.500000       15.518333
5120             1     61.708500      6    6.166667        8.405000
                 2     67.438500      6    8.000000       10.375000
                 3     71.918167      6    7.666667       13.848333
                 4     76.150333      6    8.833333       17.981667
8192             1     67.349833      6    5.166667        8.428333
                 2     75.237000      6   10.166667        9.856667
                 3     72.879167      6    8.833333       13.080000
                 4     78.637500      6   11.166667       15.931667
10240            1     68.568333      6    8.833333        9.133333
                 2     71.526000      6    8.333333        9.670000
                 3     76.504500      6   11.666667       11.088333
                 4     73.882167      6    7.166667       15.296667
20480            1     76.098000      6   11.500000       14.260000
                 2     76.242833      6   10.333333       14.260000
                 3     77.426167      6    9.833333       14.310000
                 4     77.321500      6   11.000000       14.620000

In [16]:
df.to_csv(os.path.join(os.getenv("DATA_DIR"), "mica_text_coref/movie_coref/results/coreference/cross_val_5.csv"),
          index=False)

Create cross validation data frame for hierarchical model
Variation in dev document len (2nd run)

In [37]:
results_dir = os.path.join(os.getenv("DATA_DIR"),
                           "mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_hi_Dec31")
dirs_with_no_result_file = []

for dir_ in os.listdir(results_dir):
    result_file = os.path.join(results_dir, dir_, "result.yaml")
    if not os.path.exists(result_file):
        dir__ = os.path.join(results_dir, dir_)
        print(dir__)
        dirs_with_no_result_file.append(dir__)
print()
print(f"{len(dirs_with_no_result_file)} directories have no result files")

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_hi_Dec31/Dec31_11:59:20AM_avengers_endgame_excerpts_hi_19148
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_hi_Dec31/Dec31_04:00:25PM_avengers_endgame_excerpts_hi_18756
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_hi_Dec31/Dec31_11:59:32AM_avengers_endgame_excerpts_hi_13795
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_hi_Dec31/Dec31_03:24:19PM_avengers_endgame_excerpts_hi_17946
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_hi_Dec31/Dec31_11:59:32AM_avengers_endgame_excerpts_hi_13768
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_hi_Dec31/Dec31_03:50:53PM_avengers_endgame_excerpts_hi_23712
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_hi_Dec31/Dec31_03:37:40PM_avenge

In [39]:
for dir_ in dirs_with_no_result_file:
    log_file = os.path.join(dir_, "train.log")
    with open(log_file) as fr:
        content = fr.read()
    print(dir_)
    print(content.rstrip().endswith("Testing"))
    # print(content)
    print("")

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_hi_Dec31/Dec31_11:59:20AM_avengers_endgame_excerpts_hi_19148
False

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_hi_Dec31/Dec31_04:00:25PM_avengers_endgame_excerpts_hi_18756
False

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_hi_Dec31/Dec31_11:59:32AM_avengers_endgame_excerpts_hi_13795
False

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_hi_Dec31/Dec31_03:24:19PM_avengers_endgame_excerpts_hi_17946
False

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_hi_Dec31/Dec31_11:59:32AM_avengers_endgame_excerpts_hi_13768
False

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_hi_Dec31/Dec31_03:50:53PM_avengers_endgame_excerpts_hi_23712
False

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val

In [40]:
results_dir = os.path.join(os.getenv("DATA_DIR"),
                           "mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_hi_Dec31")
epoch_pattern = re.compile(r"Epoch = (\d+)\n"
                           r"dev:: loss=[0-9\.]+, metric:Word=[0-9\.]+, Span=([0-9\.]+), Character=[0-9\.]+\n"
                           r"([0-9\.]+) GB GPU memory allocated during dev inference")
header = ["dev_document_len", "repk", "movie", "dev_lea", "best_epoch", "dev_scores", "dev_gpu_memory"]
rows = []

def search_regex(pattern, content):
    match = re.search(pattern, content)
    assert match is not None
    return match.group(1)

for dir_ in os.listdir(results_dir):
    result_file = os.path.join(results_dir, dir_, "result.yaml")
    log_file = os.path.join(results_dir, dir_, "train.log")
    with open(log_file) as fr:
        content = fr.read()
    if os.path.exists(result_file):
        with open(result_file) as f:
            result = yaml.load(f, Loader=yaml.FullLoader)
        dev_document_len = result["dev_document_len"]
        repk = result["repk"]
        movie = result["test_movie"]
        dev_lea = result["dev_metric"]["span"]["lea"]["f1"]
        epoch = result["best_epoch"]
        dev_scores = result["dev_scores"]
        dev_gpu_memory = result["dev_gpu_memory"][epoch - 1]
    else:
        if re.search(r"\nTesting", content) is not None:
            dev_document_len = int(search_regex(r"dev_document_len\s+= (\d+)", content))
            repk = int(search_regex(r"n_representative_mentions\s+= (\d+)", content))
            movie = search_regex(r"test_movie\s+= (\w+)", content)
            dev_scores, epochs, dev_gpu_memory = [], [], []
            for match in re.finditer(epoch_pattern, content):
                dev_scores.append(float(match.group(2)))
                epochs.append(int(match.group(1)))
                dev_gpu_memory.append(float(match.group(3)))
            i = np.argmax(dev_scores)
            dev_lea = dev_scores[i]
            epoch = epochs[i]
            dev_gpu_memory = dev_gpu_memory[i]
        else:
            continue
    rows.append([dev_document_len, repk, movie, dev_lea, epoch, dev_scores, dev_gpu_memory])

df = pd.DataFrame(rows, columns=header)
print(df.shape)
print()

for col, dtype in zip(df.columns, df.dtypes):
    print(f"{col} ({dtype})")

(41, 7)

dev_document_len (int64)
repk (int64)
movie (object)
dev_lea (float64)
best_epoch (int64)
dev_scores (object)
dev_gpu_memory (float64)


In [41]:
df[df["movie"] == "avengers_endgame"]

,dev_document_len,repk,movie,dev_lea,best_epoch,dev_scores,dev_gpu_memory
4,20480,5,avengers_endgame,67.904,8,"[54.7656, 55.5278, 30.723, 47.3549, 47.2504, 5...",26.56


In [9]:
macro_df = df.groupby(["dev_document_len", "repk"]).agg({"dev_lea": "mean", "movie": "count", "best_epoch": "mean",
                                                 "dev_gpu_memory": "mean"})
# macro_df = macro_df[macro_df["movie"] == 6]
macro_df

dev_lea  movie  best_epoch  dev_gpu_memory
dev_document_len repk                                              
5120             5     78.103600      5   10.600000       17.250000
                 6     77.862200      5    9.200000       21.184000
8192             5     81.141800      5   11.600000       14.296000
                 6     81.260600      5   12.200000       17.030000
10240            5     78.527400      5    9.200000       15.268000
                 6     79.144200      5    8.400000       18.680000
20480            5     76.946667      6    8.333333       17.008333
                 6     79.404400      5   10.000000       16.754000

In [ ]:
df.to_csv(os.path.join(os.getenv("DATA_DIR"), "mica_text_coref/movie_coref/results/coreference/cross_val_5.csv"),
          index=False)

Create cross validation dataframe for the less important hyperparameters

In [2]:
results_dir = os.path.join(os.getenv("DATA_DIR"),
                           "mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec30")
dirs_with_no_result_file = []

for dir_ in os.listdir(results_dir):
    result_file = os.path.join(results_dir, dir_, "result.yaml")
    if not os.path.exists(result_file):
        dir__ = os.path.join(results_dir, dir_)
        print(dir__)
        dirs_with_no_result_file.append(dir__)
print()
print(f"{len(dirs_with_no_result_file)} directories have no result files")

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec30/Dec30_06:55:20AM_avengers_endgame_excerpts_17977
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec30/Dec30_02:20:25PM_dead_poets_society_excerpts_12519
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec30/Dec31_11:55:13PM_zootopia_excerpts_27639
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec30/Dec30_11:03:52AM_dead_poets_society_excerpts_16957
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec30/Dec30_05:53:44AM_avengers_endgame_excerpts_2365
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec30/Dec31_06:24:50AM_prestige_excerpts_7317
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec30/Dec31_10:38:15PM_zootopia_excerpts_26231
/proj/sbaruah/data/mica_text_coref/

In [3]:
for dir_ in dirs_with_no_result_file:
    log_file = os.path.join(dir_, "train.log")
    with open(log_file) as fr:
        content = fr.read()
    if not content.rstrip().endswith("Testing"):
        print(dir_)
        # print(content)
        print("")

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec30/Jan01_02:45:35AM_zootopia_excerpts_31029

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec30/Jan01_02:08:42AM_zootopia_excerpts_21677

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec30/Jan01_02:31:54AM_zootopia_excerpts_30742

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec30/Jan01_02:33:48AM_zootopia_excerpts_22159

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec30/Jan01_02:29:51AM_quiet_place_excerpts_3742



In [4]:
results_dir = os.path.join(os.getenv("DATA_DIR"),
                           "mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec30")
epoch_pattern = re.compile(r"Epoch = (\d+)\n"
                           r"dev:: loss=[0-9\.]+, metric:Word=[0-9\.]+, Span=([0-9\.]+), Character=[0-9\.]+")
header = ["dev_document_len", "dev_overlap_len", "merge_strategy", "load_bert", "freeze_bert", "add_cr_to_coarse",
          "filter_by_cr", "remove_singleton", "movie", "dev_lea", "best_epoch"]
rows = []

def search_regex(pattern, content):
    match = re.search(pattern, content)
    assert match is not None
    return match.group(1)

for dir_ in os.listdir(results_dir):
    result_file = os.path.join(results_dir, dir_, "result.yaml")
    log_file = os.path.join(results_dir, dir_, "train.log")
    with open(log_file) as fr:
        content = fr.read()
    if os.path.exists(result_file):
        with open(result_file) as f:
            result = yaml.load(f, Loader=yaml.FullLoader)
        dev_document_len = result["dev_document_len"]
        dev_overlap_len = result["dev_overlap_len"]
        merge_strategy = result["dev_merge_strategy"]
        freeze_bert = result["freeze_bert"]
        add_cr_to_coarse = result["add_cr_to_coarse"]
        filter_by_cr = result["filter_mentions_by_cr"]
        remove_singleton = result["remove_singleton_cr"]
        movie = result["test_movie"]
        dev_lea = result["dev_metric"]["span"]["lea"]["f1"]
        epoch = result["best_epoch"]
    else:
        if re.search(r"\nTesting", content) is not None:
            dev_document_len = int(search_regex(r"dev_document_len\s+= (\d+)", content))
            dev_overlap_len = int(search_regex(r"dev_overlap_len\s+= (\d+)", content))
            merge_strategy = search_regex(r"dev_merge_strategy\s+= (\w+)", content)
            freeze_bert = search_regex(r"freeze_bert\s+= (\w+)", content) == "True"
            add_cr_to_coarse = search_regex(r"add_cr_to_coarse\s+= (\w+)", content) == "True"
            filter_by_cr = search_regex(r"filter_mentions_by_cr\s+= (\w+)", content) == "True"
            remove_singleton = search_regex(r"remove_singleton_cr\s+= (\w+)", content) == "True"
            movie = search_regex(r"test_movie\s+= (\w+)", content)
            dev_scores, epochs = [], []
            for match in re.finditer(epoch_pattern, content):
                dev_scores.append(float(match.group(2)))
                epochs.append(int(match.group(1)))
            i = np.argmax(dev_scores)
            dev_lea = dev_scores[i]
            epoch = epochs[i]
        else:
            continue
    load_bert = search_regex(r"load_bert\s+= (\w+)", content) == "True"
    rows.append([dev_document_len, dev_overlap_len, merge_strategy, load_bert, freeze_bert, add_cr_to_coarse,
                 filter_by_cr, remove_singleton, movie, dev_lea, epoch])

df = pd.DataFrame(rows, columns=header)
print(df.shape)
print()

for col, dtype in zip(df.columns, df.dtypes):
    print(f"{col} ({dtype})")

(306, 11)

dev_document_len (int64)
dev_overlap_len (int64)
merge_strategy (object)
load_bert (bool)
freeze_bert (bool)
add_cr_to_coarse (bool)
filter_by_cr (bool)
remove_singleton (bool)
movie (object)
dev_lea (float64)
best_epoch (int64)


In [32]:
df_ = df.copy()
df_["movie_"] = df["movie"].str.slice(0, 1)
macro_df = (df_.groupby(["dev_document_len", "dev_overlap_len", "load_bert", "freeze_bert",
                        "add_cr_to_coarse", "filter_by_cr", "remove_singleton", "merge_strategy"])
               .agg({"dev_lea": "mean", "movie_": ["count", ",".join], "best_epoch": "mean"}))
# macro_df = macro_df[macro_df["movie"] == 6]
pd.set_option("display.max_rows", 100)
macro_df

dev_lea  \
                                                                                                                           mean   
dev_document_len dev_overlap_len load_bert freeze_bert add_cr_to_coarse filter_by_cr remove_singleton merge_strategy              
5120             512             False     False       True             False        True             avg             54.886167   
                                 True      False       False            False        True             avg             60.170500   
                                                       True             False        False            avg             16.933333   
                                                                                     True             avg             58.457800   
                                                                                                      max             60.500333   
                                                                                                      min             59.871833   
                                                                                                      none            60.016500   
                                                                                                      post            59.878000   
                                                                                                      pre             60.295333   
                                                                        True         True             avg             60.697167   
                                           True        True             False        True             avg             54.464500   
                 1024            False     False       True             False        True             avg             64.262667   
                                 True      False       False            False        True             avg             69.653500   
                                                       True             False        False            avg             17.383333   
                                                                                     True             avg             69.993000   
                                                                                                      max             70.153400   
                                                                                                      min             69.975200   
                                                                                                      none            70.293333   
                                                                                                      post            70.464667   
                                                                                                      pre             70.540667   
                                                                        True         True             avg             70.444000   
                                           True        True             False        True             avg             61.554500   
                 2048            False     False       True             False        True             avg             65.106200   
                                 True      False       False            False        True             avg             72.309200   
                                                       True             False        False            avg             17.260000   
                                                                                     True             avg             71.082500   
                                                                                                      max             73.069200   
                                                                                                      min             73.083200   
                                                           

In [36]:
for index, row in macro_df.iterrows():
     if row["movie_"]["count"] < 6:
          missing_movies = set(["a","p","q","z","j","d"]).difference(set(row["movie_"]["join"].split(",")))
          # print(index, missing_movies)
          (dev_document_len, dev_overlap_len, load_bert, freeze_bert, add_cr_to_coarse, filter_by_cr,
           remove_singleton, merge_strategy) = index
          load_bert_ = "" if load_bert else "no"
          freeze_bert_ = "" if freeze_bert else "no"
          add_cr_to_coarse_ = "" if add_cr_to_coarse else "no"
          filter_by_cr_ = "" if filter_by_cr else "no"
          remove_singleton_ = "" if remove_singleton else "no"
          for movie in missing_movies:
               if movie == "z":
                    movie_ = "zootopia"
               elif movie == "q":
                    movie_ = "quiet_place"
               else: assert False
               print(f"$py --dev_document_len={dev_document_len} --dev_overlap_len={dev_overlap_len} "
                    f"--dev_merge_strategy={merge_strategy} --test_document_lens={dev_document_len} "
                    f"--test_overlap_lens={dev_overlap_len} --test_merge_strategies={merge_strategy} "
                    f"--{load_bert_}load_bert --{freeze_bert_}freeze_bert --{add_cr_to_coarse_}add_cr_to_coarse "
                    f"--{filter_by_cr_}filter_by_cr --{remove_singleton_}remove_singleton_cr "
                    f"--test_movie={movie_}")

$py --dev_document_len=5120 --dev_overlap_len=512 --dev_merge_strategy=avg --test_document_lens=5120 --test_overlap_lens=512 --test_merge_strategies=avg --load_bert --nofreeze_bert --add_cr_to_coarse --nofilter_by_cr --remove_singleton_cr --test_movie=zootopia
$py --dev_document_len=5120 --dev_overlap_len=1024 --dev_merge_strategy=avg --test_document_lens=5120 --test_overlap_lens=1024 --test_merge_strategies=avg --load_bert --nofreeze_bert --add_cr_to_coarse --nofilter_by_cr --remove_singleton_cr --test_movie=zootopia
$py --dev_document_len=5120 --dev_overlap_len=1024 --dev_merge_strategy=max --test_document_lens=5120 --test_overlap_lens=1024 --test_merge_strategies=max --load_bert --nofreeze_bert --add_cr_to_coarse --nofilter_by_cr --remove_singleton_cr --test_movie=zootopia
$py --dev_document_len=5120 --dev_overlap_len=1024 --dev_merge_strategy=min --test_document_lens=5120 --test_overlap_lens=1024 --test_merge_strategies=min --load_bert --nofreeze_bert --add_cr_to_coarse --nofilter_

In [5]:
results_dir = os.path.join(os.getenv("DATA_DIR"),
                           "mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Jan01")
dirs_with_no_result_file = []

for dir_ in os.listdir(results_dir):
    result_file = os.path.join(results_dir, dir_, "result.yaml")
    if not os.path.exists(result_file):
        dir__ = os.path.join(results_dir, dir_)
        print(dir__)
        dirs_with_no_result_file.append(dir__)
print()
print(f"{len(dirs_with_no_result_file)} directories have no result files")

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Jan01/Jan02_05:26:39PM_zootopia_excerpts_15084
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Jan01/Jan02_07:16:21AM_zootopia_excerpts_3127

2 directories have no result files


In [6]:
for dir_ in dirs_with_no_result_file:
    log_file = os.path.join(dir_, "train.log")
    with open(log_file) as fr:
        content = fr.read()
    if not content.rstrip().endswith("Testing"):
        print(dir_)
        # print(content)
        print("")

In [7]:
results_dir = os.path.join(os.getenv("DATA_DIR"),
                           "mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Jan01")
epoch_pattern = re.compile(r"Epoch = (\d+)\n"
                           r"dev:: loss=[0-9\.]+, metric:Word=[0-9\.]+, Span=([0-9\.]+), Character=[0-9\.]+")
header = ["dev_document_len", "dev_overlap_len", "merge_strategy", "load_bert", "freeze_bert", "add_cr_to_coarse",
          "filter_by_cr", "remove_singleton", "movie", "dev_lea", "best_epoch"]
rows = []

def search_regex(pattern, content):
    match = re.search(pattern, content)
    assert match is not None
    return match.group(1)

for dir_ in os.listdir(results_dir):
    result_file = os.path.join(results_dir, dir_, "result.yaml")
    log_file = os.path.join(results_dir, dir_, "train.log")
    with open(log_file) as fr:
        content = fr.read()
    if os.path.exists(result_file):
        with open(result_file) as f:
            result = yaml.load(f, Loader=yaml.FullLoader)
        dev_document_len = result["dev_document_len"]
        dev_overlap_len = result["dev_overlap_len"]
        merge_strategy = result["dev_merge_strategy"]
        freeze_bert = result["freeze_bert"]
        add_cr_to_coarse = result["add_cr_to_coarse"]
        filter_by_cr = result["filter_mentions_by_cr"]
        remove_singleton = result["remove_singleton_cr"]
        movie = result["test_movie"]
        dev_lea = result["dev_metric"]["span"]["lea"]["f1"]
        epoch = result["best_epoch"]
    else:
        if re.search(r"\nTesting", content) is not None:
            dev_document_len = int(search_regex(r"dev_document_len\s+= (\d+)", content))
            dev_overlap_len = int(search_regex(r"dev_overlap_len\s+= (\d+)", content))
            merge_strategy = search_regex(r"dev_merge_strategy\s+= (\w+)", content)
            freeze_bert = search_regex(r"freeze_bert\s+= (\w+)", content) == "True"
            add_cr_to_coarse = search_regex(r"add_cr_to_coarse\s+= (\w+)", content) == "True"
            filter_by_cr = search_regex(r"filter_mentions_by_cr\s+= (\w+)", content) == "True"
            remove_singleton = search_regex(r"remove_singleton_cr\s+= (\w+)", content) == "True"
            movie = search_regex(r"test_movie\s+= (\w+)", content)
            dev_scores, epochs = [], []
            for match in re.finditer(epoch_pattern, content):
                dev_scores.append(float(match.group(2)))
                epochs.append(int(match.group(1)))
            i = np.argmax(dev_scores)
            dev_lea = dev_scores[i]
            epoch = epochs[i]
        else:
            continue
    load_bert = search_regex(r"load_bert\s+= (\w+)", content) == "True"
    rows.append([dev_document_len, dev_overlap_len, merge_strategy, load_bert, freeze_bert, add_cr_to_coarse,
                 filter_by_cr, remove_singleton, movie, dev_lea, epoch])

df1 = pd.DataFrame(rows, columns=header)
print(df1.shape)
print()

for col, dtype in zip(df1.columns, df1.dtypes):
    print(f"{col} ({dtype})")

(24, 11)

dev_document_len (int64)
dev_overlap_len (int64)
merge_strategy (object)
load_bert (bool)
freeze_bert (bool)
add_cr_to_coarse (bool)
filter_by_cr (bool)
remove_singleton (bool)
movie (object)
dev_lea (float64)
best_epoch (int64)


In [8]:
df2 = pd.concat([df, df1])

In [9]:
print(df2.shape)
for col, dtype in zip(df2.columns, df2.dtypes):
    print(f"{col} ({dtype})")

(330, 11)
dev_document_len (int64)
dev_overlap_len (int64)
merge_strategy (object)
load_bert (bool)
freeze_bert (bool)
add_cr_to_coarse (bool)
filter_by_cr (bool)
remove_singleton (bool)
movie (object)
dev_lea (float64)
best_epoch (int64)


In [10]:
df_ = df2.copy()
df_["movie_"] = df["movie"].str.slice(0, 1)
macro_df = (df_.groupby(["dev_document_len", "dev_overlap_len", "load_bert", "freeze_bert",
                        "add_cr_to_coarse", "filter_by_cr", "remove_singleton", "merge_strategy"])
               .agg({"dev_lea": "mean", "movie_": ["count", ",".join], "best_epoch": "mean"}))
# macro_df = macro_df[macro_df["movie"] == 6]
pd.set_option("display.max_rows", 100)
macro_df

dev_lea  \
                                                                                                                           mean   
dev_document_len dev_overlap_len load_bert freeze_bert add_cr_to_coarse filter_by_cr remove_singleton merge_strategy              
5120             512             False     False       True             False        True             avg             54.886167   
                                 True      False       False            False        True             avg             60.170500   
                                                       True             False        False            avg             16.933333   
                                                                                     True             avg             60.682000   
                                                                                                      max             60.500333   
                                                                                                      min             59.871833   
                                                                                                      none            60.016500   
                                                                                                      post            59.878000   
                                                                                                      pre             60.295333   
                                                                        True         True             avg             60.697167   
                                           True        True             False        True             avg             54.464500   
                 1024            False     False       True             False        True             avg             64.262667   
                                 True      False       False            False        True             avg             69.653500   
                                                       True             False        False            avg             17.383333   
                                                                                     True             avg             70.453667   
                                                                                                      max             70.601167   
                                                                                                      min             70.417333   
                                                                                                      none            70.293333   
                                                                                                      post            70.464667   
                                                                                                      pre             70.540667   
                                                                        True         True             avg             70.444000   
                                           True        True             False        True             avg             61.554500   
                 2048            False     False       True             False        True             avg             66.166667   
                                 True      False       False            False        True             avg             72.151000   
                                                       True             False        False            avg             17.516667   
                                                                                     True             avg             73.312833   
                                                                                                      max             73.453833   
                                                                                                      min             73.382167   
                                                           

In [13]:
df2.to_csv(os.path.join(os.getenv("DATA_DIR"), "mica_text_coref/movie_coref/results/coreference/cross_val_6.csv"),
          index=False)

Create cross validation dataframe for other less important hyperparameters

In [14]:
results_dir = os.path.join(os.getenv("DATA_DIR"),
                           "mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Jan02")
dirs_with_no_result_file = []

for dir_ in os.listdir(results_dir):
    result_file = os.path.join(results_dir, dir_, "result.yaml")
    if not os.path.exists(result_file):
        dir__ = os.path.join(results_dir, dir_)
        print(dir__)
        dirs_with_no_result_file.append(dir__)
print()
print(f"{len(dirs_with_no_result_file)} directories have no result files")

for dir_ in dirs_with_no_result_file:
    log_file = os.path.join(dir_, "train.log")
    with open(log_file) as fr:
        content = fr.read()
    if not content.rstrip().endswith("Testing"):
        print(dir_)
        # print(content)
        print("")

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Jan02/Jan04_01:41:34AM_zootopia_excerpts_18761
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Jan02/Jan04_01:15:55AM_zootopia_excerpts_22109
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Jan02/Jan04_01:21:53AM_quiet_place_excerpts_18820
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Jan02/Jan03_06:22:00AM_prestige_excerpts_29431
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Jan02/Jan04_01:40:37AM_zootopia_excerpts_11380
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Jan02/Jan04_01:30:17AM_zootopia_excerpts_19022

6 directories have no result files
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Jan02/Jan04_01:41:34AM_zootopia_excerpts_18761

/proj/sbaruah/data/mica_text_

In [15]:
results_dir = os.path.join(os.getenv("DATA_DIR"),
                           "mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Jan02")
epoch_pattern = re.compile(r"Epoch = (\d+)\n"
                           r"dev:: loss=[0-9\.]+, metric:Word=[0-9\.]+, Span=([0-9\.]+), Character=[0-9\.]+")
header = ["dev_document_len", "dev_overlap_len", "genre", "bce_weight", "movie", "dev_lea", "best_epoch"]
rows = []

def search_regex(pattern, content):
    match = re.search(pattern, content)
    assert match is not None
    return match.group(1)

for dir_ in os.listdir(results_dir):
    result_file = os.path.join(results_dir, dir_, "result.yaml")
    log_file = os.path.join(results_dir, dir_, "train.log")
    with open(log_file) as fr:
        content = fr.read()
    if os.path.exists(result_file):
        with open(result_file) as f:
            result = yaml.load(f, Loader=yaml.FullLoader)
        dev_document_len = result["dev_document_len"]
        dev_overlap_len = result["dev_overlap_len"]
        genre = result["genre"]
        bce_weight = result["bce_weight"]
        movie = result["test_movie"]
        dev_lea = result["dev_metric"]["span"]["lea"]["f1"]
        epoch = result["best_epoch"]
    else:
        if re.search(r"\nTesting", content) is not None:
            dev_document_len = int(search_regex(r"dev_document_len\s+= (\d+)", content))
            dev_overlap_len = int(search_regex(r"dev_overlap_len\s+= (\d+)", content))
            genre = search_regex(r"genre\s+= (\w+)", content)
            bce_weight = float(search_regex(r"bce_weight\s+= ([0-9\.]+)", content))
            movie = search_regex(r"test_movie\s+= (\w+)", content)
            dev_scores, epochs = [], []
            for match in re.finditer(epoch_pattern, content):
                dev_scores.append(float(match.group(2)))
                epochs.append(int(match.group(1)))
            i = np.argmax(dev_scores)
            dev_lea = dev_scores[i]
            epoch = epochs[i]
        else:
            continue
    rows.append([dev_document_len, dev_overlap_len, genre, bce_weight, movie, dev_lea, epoch])

df = pd.DataFrame(rows, columns=header)
print(df.shape)
print()

for col, dtype in zip(df.columns, df.dtypes):
    print(f"{col} ({dtype})")

(328, 7)

dev_document_len (int64)
dev_overlap_len (int64)
genre (object)
bce_weight (float64)
movie (object)
dev_lea (float64)
best_epoch (int64)


In [16]:
macro_df = (df.groupby(["dev_document_len", "dev_overlap_len", "genre", "bce_weight"])
              .agg({"dev_lea": "mean", "best_epoch": "mean", "movie": "count"}))
macro_df

dev_lea  best_epoch  \
dev_document_len dev_overlap_len genre bce_weight                          
5120             512             bc    0.50        60.925333    6.166667   
                                 bn    0.50        62.362167   11.333333   
                                 mz    0.50        60.353500    6.500000   
                                 nw    0.50        61.142333    9.833333   
                                 pt    0.50        60.903500    7.666667   
                                 tc    0.50        60.374667    7.666667   
                                 wb    0.00        59.612167    6.166667   
                                       0.25        58.745000    8.000000   
                                       0.50        60.682000   10.166667   
                                       0.75        61.332500   10.000000   
                                       1.00        61.651833    7.000000   
                 1024            bc    0.50        69.256833    8.000000   
                                 bn    0.50        70.776667    9.500000   
                                 mz    0.50        69.532500    9.666667   
                                 nw    0.50        69.819333    9.500000   
                                 pt    0.50        69.032500    7.666667   
                                 tc    0.50        69.514000    8.666667   
                                 wb    0.00        68.994200    8.200000   
                                       0.25        69.567400    8.600000   
                                       0.50        69.993000    7.800000   
                                       0.75        69.411800    7.400000   
                                       1.00        70.727800    5.800000   
                 2048            bc    0.50        73.538200    8.400000   
                                 bn    0.50        73.755600    8.000000   
                                 mz    0.50        70.698200    6.000000   
                                 nw    0.50        72.882000    5.800000   
                                 pt    0.50        72.821600    7.800000   
                                 tc    0.50        72.937600   11.200000   
                                 wb    0.00        72.720400   10.000000   
                                       0.25        72.833000    8.400000   
                                       0.50        73.054400    8.000000   
                                       0.75        71.509250    7.000000   
                                       1.00        71.266250    5.250000   
8192             2048            bc    0.50        76.839833    7.666667   
                                 bn    0.50        76.615000    9.166667   
                                 mz    0.50        75.949500    4.333333   
                                 nw    0.50        75.015500    5.166667   
                                 pt    0.50        76.389833    8.000000   
                                 tc    0.50        75.585833    4.333333   
                                 wb    0.00        74.877667    8.333333   
                                       0.25        75.139000    8.166667   
                                       0.50        75.448667    6.333333   
                                       0.75        76.954500   10.500000   
                                       1.00        77.550800    6.000000   
                 3072            bc    0.50        76.791667    7.000000   
                                 bn    0.50        77.284400    5.800000   
                                 mz    0.50        76.174200    4.600000   
                                 nw    0.50        76.195000    8.200000   
                                 pt    0.50        77.415600    9.200000   
                                 tc    0.50        76.837800    6.800000   
                                 wb    0.00        76.975800   10.400000   
                         

In [17]:
macro_df.loc[(5120, 512, slice(None), slice(None))]

dev_lea  best_epoch  movie
genre bce_weight                              
bc    0.50        60.925333    6.166667      6
bn    0.50        62.362167   11.333333      6
mz    0.50        60.353500    6.500000      6
nw    0.50        61.142333    9.833333      6
pt    0.50        60.903500    7.666667      6
tc    0.50        60.374667    7.666667      6
wb    0.00        59.612167    6.166667      6
      0.25        58.745000    8.000000      6
      0.50        60.682000   10.166667     12
      0.75        61.332500   10.000000      6
      1.00        61.651833    7.000000      6

Final cross validation dataframe

In [2]:
results_dir = os.path.join(os.getenv("DATA_DIR"),
                           "mica_text_coref/movie_coref/results/coreference/final_Jan12")
dirs_with_no_result_file = []

for dir_ in os.listdir(results_dir):
    result_file = os.path.join(results_dir, dir_, "result.yaml")
    if not os.path.exists(result_file):
        dir__ = os.path.join(results_dir, dir_)
        print(dir__)
        dirs_with_no_result_file.append(dir__)
print()
print(f"{len(dirs_with_no_result_file)} directories have no result files")

for dir_ in dirs_with_no_result_file:
    log_file = os.path.join(dir_, "train.log")
    with open(log_file) as fr:
        content = fr.read()
    if not content.rstrip().endswith("Testing"):
        print(dir_)
        # print(content)
        print("")

/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/final_Jan12/Jan12_10:00:10PM_john_wick_excerpts_20363
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/final_Jan12/Jan12_01:30:02PM_avengers_endgame_excerpts_hi_13274
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/final_Jan12/Jan13_12:28:18AM_prestige_excerpts_23496
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/final_Jan12/Jan12_03:41:47PM_avengers_endgame_excerpts_12809
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/final_Jan12/Jan12_08:33:57PM_prestige_excerpts_28606
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/final_Jan12/Jan12_01:38:17PM_avengers_endgame_excerpts_hi_23831
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/final_Jan12/Jan13_06:05:22AM_zootopia_excerpts_30139
/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/final_Jan12/Jan12_01:41:22PM_avengers_endgame_excerpts_hi_2398

In [12]:
results_dir = os.path.join(os.getenv("DATA_DIR"),
                           "mica_text_coref/movie_coref/results/coreference/final_Jan12")
header = (["document_len", "overlap_len", "hierarchical", "repk", "movie"]
          + ["muc_p", "muc_r", "muc_f1"]
          + ["bcub_p", "bcub_r", "bcub_f1"]
          + ["ceafe_p", "ceafe_r", "ceafe_f1"]
          + ["conll_f1"]
          + ["lea_p", "lea_r", "lea_f1"]
          + ["max_gpu", "best_epoch"])
small_header = ["document_len", "overlap_len", "hierarchical", "repk", "movie"]
rows = []

for dir_ in os.listdir(results_dir):
    result_file = os.path.join(results_dir, dir_, "result.yaml")
    if os.path.exists(result_file):
        with open(result_file) as f:
            result = yaml.load(f, Loader=yaml.FullLoader)
        document_len = result["dev_document_len"]
        overlap_len = result["dev_overlap_len"]
        hierarchical = result["hierarchical"]
        repk = result["repk"]
        movie = result["test_movie"]

        muc_p = result["dev_metric"]["span"]["muc"]["precision"]
        muc_r = result["dev_metric"]["span"]["muc"]["recall"]
        muc_f1 = result["dev_metric"]["span"]["muc"]["f1"]

        bcub_p = result["dev_metric"]["span"]["bcub"]["precision"]
        bcub_r = result["dev_metric"]["span"]["bcub"]["recall"]
        bcub_f1 = result["dev_metric"]["span"]["bcub"]["f1"]

        ceafe_p = result["dev_metric"]["span"]["ceafe"]["precision"]
        ceafe_r = result["dev_metric"]["span"]["ceafe"]["recall"]
        ceafe_f1 = result["dev_metric"]["span"]["ceafe"]["f1"]

        lea_p = result["dev_metric"]["span"]["lea"]["precision"]
        lea_r = result["dev_metric"]["span"]["lea"]["recall"]
        lea_f1 = result["dev_metric"]["span"]["lea"]["f1"]

        conll_f1 = (muc_f1 + bcub_f1 + ceafe_f1)/3

        max_gpu = max(result["dev_gpu_memory"])
        best_epoch = result["best_epoch"]

        rows.append([document_len, overlap_len, hierarchical, repk, movie,
                     muc_p, muc_r, muc_f1,
                     bcub_p, bcub_r, bcub_f1,
                     ceafe_p, ceafe_r, ceafe_f1,
                     conll_f1,
                     lea_p, lea_r, lea_f1,
                     max_gpu, best_epoch])

df = pd.DataFrame(rows, columns=header)
print(df.shape)
print()

for col, dtype in zip(df.columns, df.dtypes):
    print(f"{col} ({dtype})")

(322, 20)

document_len (int64)
overlap_len (int64)
hierarchical (bool)
repk (int64)
movie (object)
muc_p (float64)
muc_r (float64)
muc_f1 (float64)
bcub_p (float64)
bcub_r (float64)
bcub_f1 (float64)
ceafe_p (float64)
ceafe_r (float64)
ceafe_f1 (float64)
conll_f1 (float64)
lea_p (float64)
lea_r (float64)
lea_f1 (float64)
max_gpu (float64)
best_epoch (int64)


In [19]:
macro_df = (df.groupby(["document_len", "overlap_len", "hierarchical", "repk"])
              .agg({"movie": "count", "max_gpu": ["max", "mean"], "best_epoch": "mean",
                    "muc_p": "mean", "muc_r": "mean", "muc_f1": "mean",
                    "bcub_p": "mean", "bcub_r": "mean", "bcub_f1": "mean",
                    "ceafe_p": "mean", "ceafe_r": "mean", "ceafe_f1": "mean",
                    "lea_p": "mean", "lea_r": "mean", "lea_f1": "mean",
                    "conll_f1": "mean"}))

In [20]:
_df = macro_df[macro_df["movie"]["count"] == 6]
display(_df.loc[(slice(None), slice(None), False, 3)])
display(_df.loc[(slice(None), 512, True, slice(None))])

movie max_gpu           best_epoch      muc_p  \
                         count     max      mean       mean       mean   
document_len overlap_len                                                 
2048         128             6    7.63  7.590000   9.833333  93.267167   
             256             6    7.63  7.593333   7.500000  93.427500   
             512             6    7.63  7.595000   7.500000  93.415333   
3072         256             6    7.78  7.718333   8.666667  93.155167   
             512             6    7.78  7.721667  10.666667  93.487667   
             1024            6    7.79  7.731667   9.000000  93.601167   
4096         128             6    7.91  7.826667   7.000000  93.252667   
             256             6    7.92  7.828333   6.500000  93.114167   
             512             6    7.92  7.828333   5.166667  93.285833   
             1024            6    7.92  7.833333   8.333333  93.857000   
5120         128             6    8.01  7.940000   7.500000  93.469167   
             256             6    8.06  7.948333  12.500000  93.743167   
             512             6    8.01  7.943333  11.333333  93.778667   
             1024            6    8.06  7.955000   9.500000  93.830167   
             2048            6    8.07  7.961667   8.333333  93.864000   
8192         128             6    8.55  8.533333   9.666667  93.674833   
             256             6    8.55  8.526667   8.666667  93.543333   
             512             6    8.54  8.533333   6.500000  93.732333   
             1024            6    8.55  8.535000  10.666667  93.732167   
             2048            6    8.55  8.536667   9.166667  94.076333   
10240        128             6    9.18  9.165000  11.666667  93.915333   
             256             6    9.18  9.166667   9.833333  93.841333   
             512             6    9.19  9.168333   7.000000  93.740833   
             1024            6    9.18  9.166667   7.833333  93.890167   
             2048            6    9.19  9.166667   8.500000  93.809167   

                              muc_r     muc_f1     bcub_p     bcub_r  \
                               mean       mean       mean       mean   
document_len overlap_len                                               
2048         128          90.930500  92.073500  79.905333  31.604667   
             256          90.908167  92.148500  76.921833  39.867333   
             512          91.279833  92.325333  77.815167  42.004833   
3072         256          91.191500  92.154500  78.482833  40.899167   
             512          92.148833  92.805833  78.898167  48.965833   
             1024         92.033833  92.798667  80.224500  54.154000   
4096         128          91.778500  92.496000  79.864333  42.557500   
             256          92.109333  92.595167  73.941500  50.172000   
             512          92.061500  92.654667  76.856667  47.992167   
             1024         92.084167  92.952833  82.209833  60.058667   
5120         128          91.758833  92.597833  79.632667  46.060833   
             256          91.906667  92.807000  79.406167  49.011667   
             512          91.999167  92.868333  81.929333  51.205167   
             1024         92.352167  93.078000  81.893667  63.652500   
             2048         92.830333  93.337167  81.282833  69.045500   
8192         128          91.938667  92.794333  78.752167  52.196500   
             256          92.518500  93.024333  80.432333  55.290000   
             512          92.343167  93.028000  80.250500  58.633167   
             1024         92.874333  93.289333  80.903833  67.248000   
             2048         92.469333  93.255667  81.536167  73.323000   
10240        128          92.170333  93.029833  79.472667  60.820333   
             256          92.645833  93.232333  81.958333  62.254500   
             512          92.364833  93.036833  82.005833  63.471833   
             1024         92.752333  93.310833  80.890667  71.951167   
        

movie max_gpu            best_epoch      muc_p      muc_r  \
                  count     max       mean       mean       mean       mean   
document_len repk                                                             
2048         1        6   11.95   9.153333   5.333333  94.414667  91.580333   
             2        6   24.64  14.008333   5.833333  94.394333  91.611500   
3072         1        6   11.15   8.790000   7.833333  94.358333  91.765833   
             2        6   21.54  12.646667   7.333333  94.315500  91.787500   
             3        6   38.47  18.846667   7.333333  94.282833  91.677000   
4096         1        6   10.78   8.623333   9.333333  94.302333  92.275833   
             2        6   20.13  11.995000   8.500000  94.361333  92.200500   
             3        6   35.35  17.421667   7.500000  94.269833  92.080167   
5120         1        6   10.20   8.465000   6.500000  94.276333  91.954333   
             2        6   17.93  11.253333   6.166667  94.249333  91.748500   
             3        6   30.49  15.796667   7.833333  94.167667  92.377000   
8192         1        6    9.79   8.713333   8.500000  94.124833  92.248500   
             2        6   16.36  10.573333   6.833333  94.195000  91.987500   
             3        6   27.01  14.268333   7.833333  94.279333  91.711833   
             4        6   41.73  19.543333   7.833333  94.110000  91.631500   
10240        1        6    9.42   9.178333   7.333333  94.131667  92.143667   
             2        6   14.97  10.330000   6.166667  94.021833  92.120000   
             3        6   23.93  13.313333   7.333333  93.999833  92.190167   
             4        6   36.32  17.690000   7.500000  93.943833  92.165833   

                      muc_f1     bcub_p     bcub_r    bcub_f1    ceafe_p  \
                        mean       mean       mean       mean       mean   
document_len repk                                                          
2048         1     92.950333  54.696000  77.691667  61.937000  31.852000   
             2     92.954500  64.284333  75.909500  67.754500  32.032500   
3072         1     93.014500  58.582333  76.318667  63.894833  33.079833   
             2     93.006833  70.348167  76.770000  72.340167  34.058167   
             3     92.933500  72.539167  76.186167  73.786167  33.973833   
4096         1     93.258333  66.676167  76.703667  70.059833  40.594833   
             2     93.246000  69.634833  79.271000  73.217833  37.102833   
             3     93.138333  73.111167  78.084000  75.170667  36.715833   
5120         1     93.080333  71.304667  77.501167  73.184333  34.750000   
             2     92.959333  70.052833  77.087500  72.529500  35.060833   
             3     93.240833  70.573500  78.815000  73.774333  39.478000   
8192         1     93.156667  70.541333  78.812833  73.116333  38.934667   
             2     93.054333  73.312667  78.155833  75.396000  38.741167   
             3     92.953833  75.465333  76.701333  75.812167  36.955500   
             4     92.830833  76.860167  76.132333  76.150000  33.345167   
10240        1     93.105667  71.153500  78.787667  74.019667  36.529167   
             2     93.037000  73.243833  77.818667  75.016667  34.552500   
             3     93.062167  75.707500  77.979667  76.442500  36.089833   
             4     93.022000  75.562667  77.711000  76.254667  34.838500   

                     ceafe_r   ceafe_f1      lea_p      lea_r     lea_f1  \
                        mean       mean       mean       mean       mean   
document_len repk                                                          
2048         1     49.144500  36.965000  54.451167  77.413000  61.661167   
             2     54.355833  38.959167  64.019167  75.617333  67.460667   
3072         1     50.093167  37.906500  58.338000  76.056667  63.629833   
             2     56.912333  39.602667  70.096167  76.492833  72.067167   
             3     58.772667  40.092333  72.263833  75.899000  73.500833  